# Training

# Inference

In [1]:
import joblib
from fraudetect.config import load_args_from_json
from fraudetect.preprocessing import FraudFeatureEngineer, FeatureEncoding
from fraudetect.dataset import MyDatamodule
from fraudetect.config import Arguments
import pandas as pd
from datetime import datetime, date
import os

In [2]:
run = joblib.load(
    r"D:\fraud-detection-galsen\runs-optuna\tree-models_2025-04-15_00-42_best-run.joblib"
)
results, transform_pipe, datamodule, selector = run

In [3]:
transform_pipe, datamodule, selector

(None, <fraudetect.dataset.MyDatamodule at 0x219c7a904d0>, None)

In [4]:
results

{'svc': OptunaSearchCV(cv=TimeSeriesSplit(gap=5255, max_train_size=None, n_splits=3, test_size=None),
                estimator=SVC(), max_iter=300, n_jobs=6, n_trials=500,
                param_distributions={'C': CategoricalDistribution(choices=(0.0001, 0.0008, 0.006, 0.0464, 0.3594, 2.7826, 21.5443, 166.8101, 1291.5497, 10000.0)),
                                     'class_weight': CategoricalDistribution(choices=('balanced', None)),
                                     'degree...
                                     'kernel': CategoricalDistribution(choices=('poly', 'rbf', 'linear')),
                                     'max_iter': CategoricalDistribution(choices=(10000,)),
                                     'probability': CategoricalDistribution(choices=(False,)),
                                     'random_state': CategoricalDistribution(choices=(None,)),
                                     'tol': CategoricalDistribution(choices=(0.0001,))},
                random_state=41,

In [5]:
clf = results["svc"].best_estimator_
clf

SVC(C=10000.0, max_iter=10000, tol=0.0001)

In [11]:
clf.n_features_in_

1

In [6]:
datamodule.encoder.col_transformer

ColumnTransformer(n_jobs=6, remainder='passthrough',
                  transformers=[('onehot',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 ['ProviderId', 'ChannelId', 'PricingStrategy',
                                  'Country_Currency', 'IsNight_Android',
                                  'Weekend_Channel']),
                                ('scaled',
                                 Pipeline(steps=[('standard',
                                                  StandardScaler())]),
                                 Index(['TX_AMOUNT', 'Value', 'Hour', 'DayOfWeek', 'IsWeekend', 'IsNight',
       'AccountI...
       'AccountId_ZScore_30d', 'TxnPerBatch', 'TimeBetweenBatches',
       'AccountId_MovingAvg5', 'AccountId_LongTermAvg', 'PctChangeFromAvg',
       'DailyAccountTxnCount', 'ProductFraudRate', 'ProviderFraudRate',
       'ChannelIdFraudRate'],
      dtype='object')),
                                ('cat_encode', BinaryEncoder(),
                                 ['ProductId', 'ProductCategory',
                                  'Channel_ProductCategory',
                                  'Channel_PricingStrategy', 'Provider_Product',
                                  'Hour_Channel', 'Country_Hour'])])

In [7]:
datamodule.encoder.columns_of_transformed_data

array(['onehot__ProviderId_ProviderId_1',
       'onehot__ProviderId_ProviderId_2',
       'onehot__ProviderId_ProviderId_3',
       'onehot__ProviderId_ProviderId_4',
       'onehot__ProviderId_ProviderId_5',
       'onehot__ProviderId_ProviderId_6', 'onehot__ChannelId_ChannelId_1',
       'onehot__ChannelId_ChannelId_2', 'onehot__ChannelId_ChannelId_3',
       'onehot__ChannelId_ChannelId_5', 'onehot__PricingStrategy_0',
       'onehot__PricingStrategy_1', 'onehot__PricingStrategy_2',
       'onehot__PricingStrategy_4', 'onehot__Country_Currency_256_UGX',
       'onehot__IsNight_Android_0ChannelId_1',
       'onehot__IsNight_Android_0ChannelId_2',
       'onehot__IsNight_Android_0ChannelId_3',
       'onehot__IsNight_Android_0ChannelId_5',
       'onehot__IsNight_Android_1ChannelId_1',
       'onehot__IsNight_Android_1ChannelId_2',
       'onehot__IsNight_Android_1ChannelId_3',
       'onehot__IsNight_Android_1ChannelId_5',
       'onehot__Weekend_Channel_0ChannelId_1',
       'oneho

In [8]:
args, cfg = load_args_from_json(
    r"D:\fraud-detection-galsen\runs-optuna\tree-models_2025-04-15_00-42.json"
)

In [9]:
args

Arguments(data_path='D:\\fraud-detection-galsen\\data\\training.csv', study_name='tree-models_2025-04-15_00-42', work_dir='D:\\fraud-detection-galsen\\runs-optuna', run_name='debug', delta_train=50, delta_delay=7, delta_test=20, random_state=41, windows_size_in_days=[1, 7, 30], sampler_names=None, sampler_cfgs=None, model_names=['decisionTree', 'logisticReg', 'svc'], session_gap_minutes=180, onehot_threshold=9, pyod_detectors=['abod', 'cblof', 'hbos', 'iforest', 'knn', 'loda', 'mcd', 'mo_gaal'], disable_pyod_outliers=False, disable_samplers=False, do_pca=True, do_poly_expansion=False, do_feature_selection=True, cv_n_iter=500, cv_gap=5255, cv_method='optuna', n_splits=3, n_jobs=6, scoring='f1', cat_encoding_method='binary', cat_encoding_base_n=4, cat_encoding_hash_method='md5', cat_encoding_hash_n_components=7, add_imputer=True, concat_features=None, optuna_n_trials=50, cols_to_drop=['CurrencyCode', 'CountryCode', 'SubscriptionId', 'BatchId', 'CUSTOMER_ID', 'AccountId', 'TRANSACTION_ID'

In [ ]:
# datamodule = MyDatamodule()

# feature_engineer = FraudFeatureEngineer(
#     windows_size_in_days=args.windows_size_in_days,
#     uid_cols=args.concat_features,
#     session_gap_minutes=args.session_gap_minutes,
#     n_clusters=None,
# )
# cat_encoding_kwards = dict(
#         n_components=8#args.cat_encoding_hash_n_components
# )
# encoder = FeatureEncoding(
#     cat_encoding_method=args.cat_encoding_method,
#     add_imputer=args.add_imputer,
#     onehot_threshold=args.onehot_threshold,
#     cols_to_drop=args.cols_to_drop,
#     n_jobs=args.n_jobs,
#     cat_encoding_kwards=cat_encoding_kwards,
# )

# datamodule.setup(encoder=encoder, feature_engineer=feature_engineer)

In [67]:
X_train, y = datamodule.get_predict_dataset(args.data_path)

X_pred, _ = datamodule.get_predict_dataset(
    r"D:\fraud-detection-galsen\data\test.csv"
)

step: load data
step: load data
There is no column FraudResult in loaded data.


In [68]:
X_train.shape

(95662, 101)

In [69]:
X_pred.shape

(45019, 101)

In [70]:
y_pred = clf.predict(X_pred)

ValueError: X has 101 features, but SVC is expecting 1 features as input.

In [54]:
y_pred

array([0., 0., 0., ..., 0., 0., 0.], shape=(45019,))

In [55]:
y_pred.sum(), y_pred.sum()/y_pred.shape[0]

(np.float64(79.0), np.float64(0.0017548146338212756))

In [ ]:
results, transform_pipe, col_transformer = run


for k,v in results.items():
    try:
        print(v.best_score_)
    except:
        pass

In [48]:
test_data = pd.read_csv(r"D:\fraud-detection-galsen\data\test.csv")
test_data.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy
0,TransactionId_50600,BatchId_35028,AccountId_2441,SubscriptionId_4426,CustomerId_2857,UGX,256,ProviderId_5,ProductId_3,airtime,ChannelId_3,1000.0,1000,2019-02-13T10:01:40Z,4
1,TransactionId_95109,BatchId_45139,AccountId_3439,SubscriptionId_2643,CustomerId_3874,UGX,256,ProviderId_5,ProductId_15,financial_services,ChannelId_3,2000.0,2000,2019-02-13T10:02:12Z,2
2,TransactionId_47357,BatchId_74887,AccountId_4841,SubscriptionId_3829,CustomerId_2857,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-50.0,50,2019-02-13T10:02:30Z,2
3,TransactionId_28185,BatchId_11025,AccountId_2685,SubscriptionId_4626,CustomerId_3105,UGX,256,ProviderId_5,ProductId_10,airtime,ChannelId_3,3000.0,3000,2019-02-13T10:02:38Z,4
4,TransactionId_22140,BatchId_29804,AccountId_4841,SubscriptionId_3829,CustomerId_3105,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-60.0,60,2019-02-13T10:02:58Z,2


In [49]:
# make submission
submission = pd.read_csv(r"D:\fraud-detection-galsen\data\sample_submission.csv")
submission.head()

,TransactionId,FraudResult
0,TransactionId_50600,NaN
1,TransactionId_95109,NaN
2,TransactionId_47357,NaN
3,TransactionId_28185,NaN
4,TransactionId_22140,NaN


In [ ]:
## Great same...
(test_data['TransactionId'] == submission['TransactionId']).sum()

np.int64(45019)

In [50]:
submission['FraudResult'] = y_pred

submission['FraudResult'] = submission['FraudResult'].astype('int')
submission.head()

,TransactionId,FraudResult
0,TransactionId_50600,0
1,TransactionId_95109,0
2,TransactionId_47357,0
3,TransactionId_28185,0
4,TransactionId_22140,0


In [57]:
submission['FraudResult'].sum()

np.int64(79)

In [56]:
current_time = datetime.now().strftime("%H-%M")
filename = f"submission_{str(date.today())}_{current_time}.csv"
filename = os.path.join(r"D:\fraud-detection-galsen\data",filename)

submission.to_csv(filename,index=False)